In [21]:
# Modèle théorique d'estimation des émissions sur le périphérique parisien
# Inspiré de la décomposition de Bigo (2020)

# --- Imports ---
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider



In [ ]:
# --- Paramètres par défaut ---
def default_params():
    return {
        "distance_moyenne_km": 35,
        "demande_passagers_km": 3_500_000, # 35 * 100 000 voyageurs
        "part_voitures_thermiques": 0.7,
        "part_voitures_diesel": 0.2,
        "part_voitures_elec": 0.1,
        "conso_l_100km_essence_70": 7.0,
        "conso_l_100km_essence_50": 6.5,
        "conso_l_100km_diesel_70": 5.5,
        "conso_l_100km_diesel_50": 5.0,
        "conso_kWh_100km_elec_70": 17.0,
        "conso_kWh_100km_elec_50": 15.5,
        "emission_CO2_l_essence": 2.3,
        "emission_CO2_l_diesel": 2.6,
        "emission_CO2_kWh": 0.012,
        "emission_NOx_g_km_essence": 0.25,
        "emission_NOx_g_km_diesel": 0.6,
        "emission_NOx_g_km_elec": 0.01,
        "emission_PM10_g_km_essence": 0.01,
        "emission_PM10_g_km_diesel": 0.05,
        "emission_PM10_g_km_elec": 0.005,
        "taux_remplissage_actuel": 1.2,
        "taux_remplissage_covoit": 1.8,
        "part_solo": 0.8,
        "part_covoiturage": 0.2,
        "duree_jours": 365,
        "part_temps_pointe": 0.31,
        "seuil_congestion": 100000,
        "seuil_max_congestion": 200000
    }

In [ ]:
# --- Fonction de congestion non linéaire ---
def facteur_congestion(nb_vehicules, seuil, seuil_max):
    """
    Calcule un facteur de surconsommation lié à la congestion du trafic.

    Paramètres :
    - nb_vehicules : nombre de véhicules estimés sur le tronçon
    - seuil : seuil à partir duquel la congestion commence à impacter la consommation
    - seuil_max : valeur au-delà de laquelle la congestion est très élevée

    Retourne :
    - Un facteur multiplicatif de consommation (ex. 1.0 = fluide, >1.0 = congestion)
    """

    # Si le nombre de véhicules est inférieur au seuil : trafic fluide
    if nb_vehicules <= seuil:
        return 1.0

    # Si entre le seuil et le seuil max : congestion modérée (facteur augmente progressivement)
    elif nb_vehicules <= seuil_max:
        # Interpolation linéaire entre 1.0 et 1.1
        return 1.0 + 0.1 * (nb_vehicules - seuil) / (seuil_max - seuil)

    # Si au-dessus du seuil max : congestion forte (facteur augmente au-delà de 1.1)
    else:
        return 1.1 + 0.1 * (nb_vehicules - seuil_max) / seuil_max

In [ ]:
# --- Fonction d'estimation des émissions ---
def calcul_emissions(params, vitesse_reduite=False, part_temps=1.0, voie_covoit_activee=False):
    """
    Estime les émissions de CO2, NOx et PM10 selon les paramètres donnés.

    - Si la voie de covoiturage est activée et qu'on est aux heures de pointe,
      on distingue deux groupes de véhicules (prioritaires et classiques).
    - Sinon, tous les véhicules sont traités de manière homogène.

    Paramètres :
    - params : dictionnaire contenant tous les paramètres du modèle
    - vitesse_reduite : booléen, si la vitesse est réduite (50 km/h au lieu de 70)
    - part_temps : part du temps journalier considéré (ex : 0.31 pour les heures de pointe)
    - voie_covoit_activee : booléen, True si la voie réservée est active
    """

    # --- Paramètres de base ---
    d = params["distance_moyenne_km"]                   # distance moyenne par véhicule
    D = params["demande_passagers_km"]                  # demande totale (en pass.km)
    jours = params["duree_jours"]                       # nombre de jours considérés

    # Centré sur les personnes (80% voyagent seuls)
    r = 1 / (params["part_solo"] / 1.0 + params["part_covoiturage"] / params["taux_remplissage_covoit"]) #combien de personnes en moyenne transp. par 1 véhicule basé sur comportement des gens

    # Nombre total de véhicules requis
    n = D / (r * d)
    print(f"\n>>> Nombre de véhicules ajusté : {n:.0f}")

    # --- Cas spécial : voie de covoiturage active aux heures de pointe ---
    if voie_covoit_activee and part_temps == params["part_temps_pointe"]:
        # Répartition du trafic en 2 groupes :
        part_prioritaire = 0.2   # véhicules pouvant emprunter la voie réservée
        part_classique = 0.8     # les autres

        n_prioritaire = n * part_prioritaire
        n_classique = n * part_classique

        # Facteurs de congestion spécifiques à chaque groupe
        congestion_p = 1.02  # congestion très faible pour les prioritaires
        congestion_c = facteur_congestion(n_classique, params["seuil_congestion"], params["seuil_max_congestion"])

        result = {"CO2": 0, "NOx": 0, "PM10": 0}

        # Boucle sur les types de motorisation
        for type_v, part in [
            ("essence", params["part_voitures_thermiques"]),
            ("diesel", params["part_voitures_diesel"]),
            ("elec", params["part_voitures_elec"])
        ]:
            # Sélection des paramètres techniques
            if type_v == "essence":
                conso = params["conso_l_100km_essence_50"] if vitesse_reduite else params["conso_l_100km_essence_70"]
                CO2e = params["emission_CO2_l_essence"]
                NOx = params["emission_NOx_g_km_essence"]
                PM = params["emission_PM10_g_km_essence"]
            elif type_v == "diesel":
                conso = params["conso_l_100km_diesel_50"] if vitesse_reduite else params["conso_l_100km_diesel_70"]
                CO2e = params["emission_CO2_l_diesel"]
                NOx = params["emission_NOx_g_km_diesel"]
                PM = params["emission_PM10_g_km_diesel"]
            else:  # elec
                conso = params["conso_kWh_100km_elec_50"] if vitesse_reduite else params["conso_kWh_100km_elec_70"]
                CO2e = params["emission_CO2_kWh"]
                NOx = params["emission_NOx_g_km_elec"]
                PM = params["emission_PM10_g_km_elec"]

            # Calcul séparé pour chaque groupe
            for part_groupe, n_groupe, congestion in [
                (part_prioritaire, n_prioritaire, congestion_p),
                (part_classique, n_classique, congestion_c)
            ]:
                km_total = part * n_groupe * d * part_temps
                consommation_totale = km_total * (conso / 100) * congestion
                result["CO2"] += consommation_totale * CO2e / 1000 * jours
                result["NOx"] += km_total * NOx / 1_000_000 * jours
                result["PM10"] += km_total * PM / 1_000_000 * jours

        return result

    # --- Cas classique : tous les véhicules traités ensemble ---
    else:
        congestion = facteur_congestion(n, params["seuil_congestion"], params["seuil_max_congestion"])

    result = {"CO2": 0, "NOx": 0, "PM10": 0}

    for type_v, part in [
        ("essence", params["part_voitures_thermiques"]),
        ("diesel", params["part_voitures_diesel"]),
        ("elec", params["part_voitures_elec"])
    ]:
        if type_v == "essence":
            conso = params["conso_l_100km_essence_50"] if vitesse_reduite else params["conso_l_100km_essence_70"]
            CO2e = params["emission_CO2_l_essence"]
            NOx = params["emission_NOx_g_km_essence"]
            PM = params["emission_PM10_g_km_essence"]
        elif type_v == "diesel":
            conso = params["conso_l_100km_diesel_50"] if vitesse_reduite else params["conso_l_100km_diesel_70"]
            CO2e = params["emission_CO2_l_diesel"]
            NOx = params["emission_NOx_g_km_diesel"]
            PM = params["emission_PM10_g_km_diesel"]
        else:
            conso = params["conso_kWh_100km_elec_50"] if vitesse_reduite else params["conso_kWh_100km_elec_70"]
            CO2e = params["emission_CO2_kWh"]
            NOx = params["emission_NOx_g_km_elec"]
            PM = params["emission_PM10_g_km_elec"]

        km_total = part * n * d * part_temps
        consommation_totale = km_total * (conso / 100) * congestion
        CO2_total = consommation_totale * CO2e / 1000 * jours
        NOx_total = km_total * NOx / 1_000_000 * jours
        PM10_total = km_total * PM / 1_000_000 * jours

        result["CO2"] += CO2_total
        result["NOx"] += NOx_total
        result["PM10"] += PM10_total

    return result

In [ ]:
# --- Fusion pointe / hors-pointe ---
def emissions_horaires_combinees(params, vitesse_reduite=False, voie_covoit_activee=False):
    """
    Calcule les émissions totales en combinant les périodes de pointe et hors-pointe.

    Paramètres :
    - params : dictionnaire contenant tous les paramètres du modèle
    - vitesse_reduite : booléen, True si la vitesse est réduite à 50 km/h
    - voie_covoit_activee : booléen, True si la voie de covoiturage est activée pendant les heures de pointe

    Retour :
    - Dictionnaire avec les émissions annuelles totales de CO2, NOx et PM10 (en tonnes)
    """

    # --- Répartition du temps ---
    part_pointe = params["part_temps_pointe"]  # part du temps journalier en heure de pointe (ex. 0.31)
    part_off = 1 - part_pointe                 # part du temps en heure creuse (hors-pointe)

    # --- Ajustement de la surconsommation aux heures de pointe ---
    # (ancien système — plus utilisé ici, mais laissé par sécurité)
    surconso_pointe = params["surconso_congestion_pointe"]
    if voie_covoit_activee:
        # Si la voie covoiturage est active, on réduit un peu la surconsommation estimée
        surconso_pointe = min(1.05, surconso_pointe - 0.05)

    # --- Calcul des émissions pour les heures de pointe ---
    res_pointe = calcul_emissions(
        params,
        vitesse_reduite=vitesse_reduite,
        part_temps=part_pointe,
        voie_covoit_activee=voie_covoit_activee  # on active la logique des 2 groupes si demandé
    )

    # --- Calcul des émissions pour les heures hors-pointe ---
    res_off = calcul_emissions(
        params,
        vitesse_reduite=vitesse_reduite,
        part_temps=part_off,
        voie_covoit_activee=False  # la voie covoiturage est inactive hors-pointe
    )

    # --- Somme des émissions totales sur l’année ---
    total = {
        k: res_pointe[k] + res_off[k] for k in res_pointe
    }
    return total


In [26]:
# --- Affichage ---
def afficher_resultats(params):
    scenarios = {
        "A. 70 km/h sans voie covoiturage": emissions_horaires_combinees(params, vitesse_reduite=False, voie_covoit_activee=False),
        "B. 50 km/h sans voie covoiturage": emissions_horaires_combinees(params, vitesse_reduite=True, voie_covoit_activee=False),
        "C. 50 km/h avec voie covoiturage": emissions_horaires_combinees(params, vitesse_reduite=True, voie_covoit_activee=True)
    }

    for nom, valeurs in scenarios.items():
        print(f"\n{nom} :")
        for polluant, val in valeurs.items():
            print(f"  {polluant} : {val:.2f} t/an")

    labels = list(scenarios.keys())
    CO2_vals = [scenarios[k]["CO2"] for k in labels]
    NOx_vals = [scenarios[k]["NOx"] for k in labels]
    PM10_vals = [scenarios[k]["PM10"] for k in labels]

    x = np.arange(len(labels))
    width = 0.25

    fig, axes = plt.subplots(1, 3, figsize=(15, 4))

    # CO2
    axes[0].bar(labels, CO2_vals, color='green')
    axes[0].set_title("Émissions de CO₂")
    axes[0].set_ylabel("t/an")
    axes[0].tick_params(axis='x', rotation=20)

    # NOx
    axes[1].bar(labels, NOx_vals, color='orange')
    axes[1].set_title("Émissions de NOx")
    axes[1].tick_params(axis='x', rotation=20)

    # PM10
    axes[2].bar(labels, PM10_vals, color='gray')
    axes[2].set_title("Émissions de PM10")
    axes[2].tick_params(axis='x', rotation=20)

    plt.tight_layout()
    plt.show()

In [27]:
# --- Interface interactive ---
@interact(
    distance_moyenne_km=FloatSlider(min=10, max=50, step=1, value=35, description="km/jour"),
    part_covoiturage=FloatSlider(min=0.0, max=0.5, step=0.01, value=0.2, description="% covoiturage"),
    surconso_congestion_pointe=FloatSlider(min=1.0, max=1.2, step=0.01, value=1.1, description="Surconso pointe"),
    surconso_congestion_offpointe=FloatSlider(min=1.0, max=1.2, step=0.01, value=1.0, description="Surconso hors-pointe"),
    seuil_congestion=FloatSlider(min=50000, max=200000, step=5000, value=100000, description="Seuil congestion"),
    seuil_max_congestion=FloatSlider(min=100000, max=300000, step=5000, value=200000, description="Seuil max congestion")
)
def simulation_interactive(distance_moyenne_km, part_covoiturage,
                           surconso_congestion_pointe, surconso_congestion_offpointe,
                           seuil_congestion, seuil_max_congestion):
    params = default_params()
    params.update({
    "distance_moyenne_km": distance_moyenne_km,
    "part_covoiturage": part_covoiturage,
    "surconso_congestion_pointe": surconso_congestion_pointe,
    "surconso_congestion_offpointe": surconso_congestion_offpointe,
    "seuil_congestion": seuil_congestion,
    "seuil_max_congestion": seuil_max_congestion
})
    afficher_resultats(params)

interactive(children=(FloatSlider(value=35.0, description='km/jour', max=50.0, min=10.0, step=1.0), FloatSlide…